This lab is based on a lab from the ECE 417 course of the University of Illinois: https://courses.engr.illinois.edu/ece417/fa2017/mp5.html

# 1 Audiovisual Speech Recognition

## 1.1 Our goals today:

This lab will introduce you to the basics of automatic speech recognition using audio features only, image features only, and the combination of both features, i.e., audiovisual speech recognition. The speech recognition system is based on Hidden Markov Models. We will implement part of the training and decoding algorithm. You will train and test the automatic speech recognition system using the "leave-one-out" scheme. You will learn to understand the importance of audio, visual, and audiovisual features for speech recognition. 


## 1.2 Data <i class="fa fa-file"></i>
The data that you will use corresponds to 10 instances of the digit "2" being spoken and 10 instances of the digit "5" being spoken.
The audio data consists of MFCC features and the visual data consists of lip tracking features. 
These features can be found in the data/ folder.
Since there are not a lot of instances, there is no split of training and test data. In order to evaluate the performance of your speech recognizer you will be making use of cross validation, specifically the "leave one out" scheme, instead of a typical test set. 

You can read data/README.md for more details on the data.

1. Inspect the data. What does the data look like? Describe what the rows and columns represent.

2. Compare the audio features and video features of the same instance. What is similar? What is different?

3. Now take a look at other instances. What is similar? What is different?


## 1.3 Hidden Markov Model
The method that you will use to develop your bi-modal speech recognizer is the Hidden Markov Model (HMM):

"
> The HMM is based on augmenting the Markov chain. A Markov chain is a model that  tells  us  something  about  the  probabilities  of  sequences  of  random  variables, states, each of which can take on values from some set. These sets can be words, or tags, or symbols representing anything, like the weather.  A Markov chain makes a very strong assumption that if we want to predict the future in the sequence, all that matters is the current state. The states before the current state have no impact on the future except via the current state. It’s as if to predict tomorrow’s weather you could examine today’s weather but you weren’t allowed to look at yesterday’s weather.

> A Markov chain is useful when we need to compute a probability for a sequence of observable events. In many cases, however, the events we are interested in are hidden: we don’t observe them directly. For example we don’t normally observe speech units in an audio signal. Rather must infer the speech units from the sequence of audio. We call the speech units hidden because they are not observed. A hidden Markov model(HMM) allows us to talk about both observed events (like an audio sequence) and hidden events (like speech units) that we think of as causal factors in our probabilistic model.

> An HMM $W = (A,B)$ is specified by the following components:
* $Q=q_1q_2...q_N$ a set of $N$ states
* $A=A_{11}...a_{ij}...a_{NN}$ a **transition probability matrix** $A$, each $a_{ij}$ representing the probability of moving from state $i$ to state $j$, s.t. $\sum^N_{j=1}a_{ij}=1$  $\forall i$
* $O=o_1o_2...o_T$ a sequence of $T$ **observations**, each one drawn from a vocabulary $V=v_1,v_2,...,v_V$
* $B=b_i(o_t)$ a sequence of **observation likelihoods**, also called **emission probabilities**, each expressing the probability of an obervation $o_t$ being generated from state $i$
* $\pi=\pi_1,\pi_2,...,\pi_N$ an **initial probability distribution** over states. $\pi_i$ being the probability that the Markov chain will start in state $i$. Some states $j$ may have $\pi_j=0$, meaning that they cannot be initial states. Also, $\sum^n_{i=1}pi_i=1$

"

\- Speech and Language Processing.  Daniel Jurafsky & James H. Martin. Chapter A.

This lab focuses on two of the fundamental HMM problems:

*  Given an observation sequence $O$ and the set of states in the HMM, learn the HMM probabilities $A$ and $B$, which will allow us to compute the class conditional probability $P(O|W)$, i.e., the acoustic model. In order to train the HMM acoustic models we will use a version of the Expectation-Maximization algorithm called the Forward-Backward algorithm.
*  Given an observation sequence $O$ consisting of a sequence of features (audio only, visual only, audio-visual), determine the best-matching digit $P(W|O)$. For testing, we use the Forward algorithm, which is very close to the Viterbi algorithm discussed in class, except that it does not take the max over the history but rather sums over the history.

You are provided with code that implements the Expectation-Maximization algorithm.
The code is only partially complete however. It is up to you to fill in the gaps.


# 2 Code

In [ ]:
# Imports
import numpy as np

## 2.1 Training the HMM acoustic model, P(O|W)

For training the HMM acoustic models we will use the Expectation-Maximization (EM) algorithm (see Figure 6.16 of the book). 

The EM algorithm consists of the following steps:

1. Initialize $A$ and $B$
2. Expectation step:
  1. Forward-Backward algorithm:
      1. Calculate the forward probability $\alpha$ (Forward step)
      2. Calculate the backward probability $\beta$ (Backward step)
  2. Calculate the state occupancy count $\gamma$ (number of times a state is occupied)
  3. Calculate the state transition count $\xi$ (number of transitions from a state)
3. Maximization step
  1. Calculate $A$ using $\xi$
  2. Calculate $B$ using $\gamma$
4. Repeat steps 2 and 3 until convergence
5. Return $A$ and $B$
  
The implementation of the EM algorithm below deviates a little bit of the algorithm that is described in the book, but the overal picture is the same.

ghmm_train is the function that you will call to train an HMM.

Once the acoustic models are trained, these will be used to calculate the class conditional probabilities $P(O|W)$ which we need to determine the most likely digit during testing.

In [ ]:
def ghmm_train(Yseq,N,Ainit=None):
# Input:
#  N - number of states
#  Ainit  - This is the initial A matrix that you can input to force the model
#           to train a left to right model.
#  Yseq - This is the data. 

# Output:
#  P0 - Initial state probability matrix
#  A - Transition probability matrix
#  mu - Mean matrix
#  sigma - Covariance matrix 

    M = np.shape(Yseq[0])[1]
    P0 = np.ones([N,1])/N;
    
    if Ainit is None:
        A = np.random.rand(N,N) + 2 * np.eye(N)
        for l in range(N):
            A[l,:] = A[l,:]/np.sum(A[l,:]);
    else:
        A = Ainit
        
    Nseq = len(Yseq)
    datatmp = Yseq[0]
    
    for seq in range(1,Nseq):
        datatmp = np.append(datatmp,Yseq[seq],axis=0)
        
    M = np.shape(datatmp)[1]
    T = np.shape(datatmp)[0]
    T1 = int(np.floor(T/N))
    
    mu = np.zeros([N,M])
    sigma = np.zeros([N,M,M])

    for i in range(N):
        mu[i,:] = np.mean(datatmp[i*T1:(i+1)*T1-1,:],axis=0)
        sigma[i,:,:] = np.cov(np.transpose(datatmp))
        
    for itr in range(25):
        params = ghmm_em_obs(Yseq,N,A,P0,mu,sigma)
        mu = params[0]
        sigma = params[1]
        A = ghmm_em_trans(Yseq,N,A,P0,mu,sigma)
    return(P0,A,mu,sigma)

## <i class="fa fa-file"></i>
1. How are the transition probabilities of the HMM (A) initialized?
2. How are the means and covariances of the HMM (mu and sigma, used to calculated B) initialized?
3. What is the last for loop doing? Look at the other functions to understand the training process. What do you think would happen if you repeat this for loop only one time instead of 25 times? What would happen if you repeat it one million times?

ghmm_em_obs calculates the observation mean and covariances. These are used to calculate the emmission probabilities.

Since our observations are multi-dimensional vectors, we need multivariate Gaussians in order to be able to assign a probability to a multi-dimensional vector. A multivariate Gaussian is defined by a mean vector $m$ of dimensionality $D$ and a covariance matrix $S$(igma).
During training we need to estimate the mean vector $m$ and the covariance matrix $S$. In practice however, we simply keep a mean and variance for each dimension. 

You don't need to do anything yourself with this function other than running it once.

In [ ]:
def ghmm_em_obs(Yseq,N,A,P0,mu,sigma):
    N_seq = len(Yseq)
    eps = 1e-5
    M = np.shape(Yseq[0])[1]
    museq_unnorm = np.zeros([N,M])
    museq_norm = np.zeros([N,M])
    sigmaseq_unnorm = np.zeros([N,M,M])
    sigmaseq_norm = np.zeros([N,1])
    Pxyseq = list([] for _ in range(N_seq))
    Pxxyseq = list([] for _ in range(N_seq))
    llseq = list([] for _ in range(N_seq))
    
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]

        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        Pxyseq[seq] = Pxy
        Pxxyseq[seq] = Pxxy
        llseq[seq] = np.exp(np.sum(scale))
        
        for i in range(N):
            Pxy0 = Pxy[:,i].reshape(-1,1)
            Pxy0 = np.sum(np.multiply(np.repeat(Pxy0,M,axis=1),Y),axis=0)+eps
            museq_unnorm[i,:] = np.add(museq_unnorm[i,:],Pxy0)
            museq_norm[i,:] = np.add(museq_norm[i,:],np.sum(Pxy[:,i])+eps)
            
    mu = np.divide(museq_unnorm,museq_norm)
    sigmaseq_unnorm = np.zeros([N,M,M])
    sigmaseq_norm = np.zeros([N,1])
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]

        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        Pxyseq[seq] = Pxy
        Pxxyseq[seq] = Pxxy
        llseq[seq] = np.exp(np.sum(scale));
    
        for i in range(N):
            Pxy0 = Pxy[:,i]
            sigmatmp = np.zeros([M,M])
            for t in range(T):
                sigmatmp = np.add(sigmatmp, np.multiply(np.transpose(Y[t,:] - mu[i,:]).reshape(-1,1), np.multiply(Y[t,:] - mu[i,:],Pxy0[t])))
                sigmaseq_norm[i] = sigmaseq_norm[i] + Pxy0[t]
            sigmaseq_unnorm[i,:,:] = sigmaseq_unnorm[i,:,:] + np.reshape(sigmatmp,[1,M,M],order='F')
            
    for i in range(N):
        sigma[i,:,:] = np.divide(sigmaseq_unnorm[i,:,:],sigmaseq_norm[i])
    return(mu,sigma)                                              

### 2.1.1 Expectation step <i class="fa fa-file"></i>
This is the expectation step of the EM algorithm. Here ghmm_prob calculates two probabilities:

First is the probability of being in state $S_i$ at time $t$, given the observation sequence $O$ and the model $W$:

$\gamma_t(i)= P(q_t=S_i|O,W) = \frac{\alpha_t(i)\beta_t(i)}{P(O|W)} = \frac{\alpha_t(i)\beta_t(i)}{\sum_{i=1}^N\alpha_t(i)\beta_t(i)}$

You will need to insert a line of code into this function that computes $\gamma_t(i)$ ( i.e. Pxy ) when you are given $\alpha_t(i)\beta_t(i)$. Put that code in your Google Document. 

This probability is used by ghmm_em_obs to calculate the observation mean and covariances, which are used to calculated the emission probabilities.

Second is the probability of being in state $S_i$ at time $t$ and $S_j$ at time $t+1$, given the observation sequence $O$ and the model $W$:

$\xi_t(i,j)= P(q_t=S_i,q_{t+1}=S_j|O,W) = \frac{\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)}{P(O|W)} = \frac{\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)}{\sum_{i=1}^N\sum_{j=1}^N\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)}$

 You will need to insert another line of code into this function that computes $\xi_t(i,j)$ ( i.e. Pxxy ) when you are given $\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)$. 
 
 Put the code in your Google Document. 

This probability is used by ghmm_em_trans to calculate the transition probabilities between HMM states.



In [ ]:
def ghmm_prob(alpha,beta,scale,A,mu,sigma,Y):
# Input:
#   alpha - forward probability (explained in 2.1.2)
#   beta - backward probability (explained in 2.1.3)
#   Y - observation vector sequence 
#   A - state transition probabilities
#   P0 - initial state probabilities 
#   mu - observation mean
#   sigma - observation variance

# Output:
#  Pxy = P(x_t|Y)
#  Pxxy = P(x_t,x_{t+1}|Y)

    scale = np.exp(scale)
    N = np.shape(A)[0]
    i = np.ones([1,N])
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    
    Pys = np.zeros([N,T])

    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],(M,M),order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))
    Pxy = np.zeros([T,N])
    for t in range(T):
        Pxy0 = np.multiply(alpha[:,t],np.exp(beta[:,t])*scale[t])
        Pxy[t,:] = ???
        
    Pxxy = np.zeros([T,N,N])
    for t in range(T-1):
        Pxxy0 = np.multiply(alpha[:,t].reshape(-1,1) * np.transpose(np.multiply(np.exp(beta[:,t+1]),Pys[:,t+1])),A)
        Pxxy[t,:,:]= ???
    return(Pxy,Pxxy)

### 2.1.2 Forward step <i class="fa fa-file"></i>

This is the forward step of the forward-backward algorithm. It calculates the forward probability:

$\alpha_t(i)=P(O_1O_2...O_t,q_t=S_i|W)$

which is the probability of a partial observation sequence $O_1O_2...O_t$ and state $S_i$ at time $t$, given the model $W$.

You will need to insert a line of code into ghmm_fwd that computes the forward probability (i.e. alpha\_t) from the transition probabilities A, emission probabilities Pys and the previous forward probability alpha[:,t-1]:

$\alpha_{t}(j) = [\sum^N_{i=1} \alpha_{t-1}(i)a_{ij}] b_j (O_{t}), 1<t\leq T-1, 1\leq j \leq N$

The initialization $\alpha_1(i) = \pi_1b_i(O_1),1\leq i\leq N$ has already been done.

Put the code in your Google Document.

Once you have trained an HMM you can use this function to determine the likelihood of an observation sequence belonging to that HMM.

In [ ]:
def ghmm_fwd(Y,A,P0,mu,sigma):
# Input:
#   Y - observation vector sequence
#   A = state transition probabilities
#   P0 - initial state probabilities
#   mu - observation mean
#   sigma - observation variance

# Output:
#   alpha - forward probability (explained in 2.1.2)
#   scale - log Pr( O_1, O_2, ..., O_t | W), 1 < t <= T
    N = np.shape(A)[0]
    i = np.ones([1,N])
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    alpha = np.zeros([np.shape(P0)[0],T])
    Pys = np.zeros([N,T])
    
    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],(M,M),order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))

    alpha0 = np.multiply(P0,Pys[:,0].reshape(-1,1))
    scale = np.zeros([T,1])
    scale[0] = np.sum(alpha0)    
    alpha[:,0] = (alpha0 / scale[0]).flatten()
    
    for t in range(1,T):
        # Hint: A 1-d numpy vector is typically (1,n) and cannot be transposed. 
        #  You can use .reshape(-1,1) to make it (n,1). 
        #  For matrices you can simply use np.transpose.
        alpha_t = ???
        scale[t] = np.sum(alpha_t)
        alpha[:,t] = alpha_t / scale[t]
        
    scale = np.log(scale)
    return (alpha,scale)

### 2.1.3 Backward step <i class="fa fa-file"></i>

This is the backward step of the forward-backward algorithm. It calculates the backward probability:

$\beta_t(i) = P(O_{t+1}O_{t+2}...O_T|q_t=S_i,W)$

which is the probability of the partial observation sequence from $t+1$ to $T$, given state $S_i$ at time $t$ and the model $W$

You will need to insert a line of code into ghmm_bwd that computes the current backward probability (i.e. beta[:,t]) from the transition probabilities A, emission probabilities Pys, and the previous backward probability beta[:,t+1]:

$\beta_t(i) = \sum^N_{j=1}a_{ij}b_j(O_{t+1})\beta_{t+1}(j),t=T-1,T-2,...,1, 1 \leq i \leq N$

The initialization $\beta_T(i) = 1,1\leq i\leq N$ has already been done.

Put the code in your Google Document.

In [ ]:
def ghmm_bwd(Y,A,P0,mu,sigma,scale):
# Input:
#   T - observation vector sequence
#   A = state transition probabilities
#   P0 - initial state probabilities
#   mu - observation mean
#   sigma - observation variance
#   scale - log Pr( O_1, O_2, ..., O_t | W), 1 < t <= T (see 'ghmm_fwd')

# Output:
#   beta - backward probability (explained in 2.1.3)

    scale = np.exp(scale)
    N = np.shape(A)[0];
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    Pys = np.zeros([N,T])
    
    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],[M,M],order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))
            
    beta = np.zeros([N,T])
    beta[:,T-1] = (np.ones([N,1])/scale[T-1]).flatten()
    for t in range(T-2,-1,-1):
        beta[:,t] = ???
        beta[:,t] = beta[:,t] / scale[t]
    beta = np.log(beta+1.0e-10)
    
    return beta

### 2.1.4 Maximization step <i class="fa fa-file"></i>

This is the maximization step of the EM algorithm. Here ghmm_em_trans trains the probabilities of the transition matrix.

You will need to insert a line of code into this function that computes the new transition probabilities A[k,:] from the corresponding soft-count matrix Naseq[k,:]:

$\hat{a}_{ij} = \frac{\textrm{Expected number of transitions from state S_i to state S_j}}{\textrm{Expected number of transitions from state S_i}} = \frac{\sum_{t=1}^{T-1} \xi_t(i,j)}{\sum_{t=1}^{T-1}\sum_{k=1}^N \xi_t(i,k)}$

Put the code in your Google Document.

These transition probabilities are trained over several iterations and are needed in order to determine the likelihood of an obervation belonging to an HMM.

In [ ]:
def ghmm_em_trans(Yseq,N,A,P0,mu,sigma):
    N_seq = np.max(np.shape(Yseq))
    MAX_ITR = 20
    eps=0
    Naseq = np.zeros([N,N])
    
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]
        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        
        Na = np.zeros([N,N])
        for i in range(N):
            for j in range(N):
                Na[i,j] = np.sum(Pxxy[:,i,j])
        Na = Na+eps
        Naseq = Naseq + Na
    Anew = np.zeros([N,N])
    for k in range(N):
        Anew[k,:] = ???
    return Anew

# 3 Test an unknown input, argmax P(W|O) <i class="fa fa-file"></i>
Now that you can train an HMM, you can use its parameters with ghmm_fwd to predict the likelihood of a sequence belonging to that HMM. Make sure you read the description of the $scale$ variable.

You now have to evaluate how well the HMMs can distinguish between the utterances of the digit "2" and of the digit "5". Since there is no testing data, you will instead make use of the "leave one out" scheme. 

There are a total of 20 utterances. Train an HMM for the digit "2" and another for the digit "5" using 19 utterances of the data and test it on the single utterance that you have left out.  Repeat this 20 times until you have tested all 20 utterances. 

Do this procedure for:
* Only the video features
* Only the audio features
* The audio and video features combined (concatenated).

For each of these sets of features, determine the performance (separately) by calculating the accuracy: $\frac{\#correct}{20}$. Put the three accuracies in your document. Include the "leave one out" code as well.

Please note: Depending on your machine and system, the audio features can take a long time to train on. Running the code in the VM that is provided for this course can significantly speed up the training. 

In [ ]:
path = "data"
N = 5
Ainit = np.array([[0.8,0.2,0,0,0],[0,0.8,0.2,0,0],[0,0,0.8,0.2,0],[0,0,0,0.8,0.2],[0,0,0,0,1]])

# Enter your code here

## <i class="fa fa-file"></i>

1. Which type of data performs better? 
2. What do you think could be a reason for that?
